In [6]:
import cv2

# Path ke video yang akan diekstrak
video_path = "Road traffic video for object recognition - short.mp4"  # Ganti dengan path video Anda
output_frame_path = "traffic_2_direction.jpg"  # Path untuk menyimpan frame pertama

# Membuka video
cap = cv2.VideoCapture(video_path)

# Mengecek apakah video berhasil dibuka
if not cap.isOpened():
    print("Error: Tidak dapat membuka video.")
else:
    # Membaca frame pertama
    ret, frame = cap.read()
    if ret:
        # Menyimpan frame pertama sebagai gambar
        cv2.imwrite(output_frame_path, frame)
        print(f"Frame pertama berhasil diekstrak dan disimpan di: {output_frame_path}")
    else:
        print("Error: Tidak dapat membaca frame pertama.")

# Menutup video
cap.release()


Frame pertama berhasil diekstrak dan disimpan di: traffic_2_direction.jpg


In [13]:
import cv2
import numpy as np

# Load the image
image_path = output_frame_path
image = cv2.imread(image_path)
if image is None:
    raise ValueError("Image not found. Please check the image path.")


In [14]:
# Variables for drawing rectangles
start_point = None
end_point = None
is_drawing = False
rectangles = []

# Track window state
is_minimized = False

# Scale display settings
window_scale = 0.5  # Scale factor for display, adjust as needed
scaled_image = cv2.resize(image, (0, 0), fx=window_scale, fy=window_scale, interpolation=cv2.INTER_LINEAR)


def draw_rectangle(event, x, y, flags, param):
    global start_point, end_point, is_drawing, rectangles

    # Scale coordinates back to original image size
    scale_back_x = 1 / window_scale
    scale_back_y = 1 / window_scale

    if event == cv2.EVENT_LBUTTONDOWN:
        # Start drawing
        start_point = (int(x * scale_back_x), int(y * scale_back_y))
        is_drawing = True

    elif event == cv2.EVENT_MOUSEMOVE and is_drawing:
        # Update endpoint as the mouse moves
        end_point = (int(x * scale_back_x), int(y * scale_back_y))
        temp_image = image.copy()
        cv2.rectangle(temp_image, start_point, end_point, (0, 0, 255), 2)
        temp_scaled = cv2.resize(temp_image, (0, 0), fx=window_scale, fy=window_scale, interpolation=cv2.INTER_LINEAR)
        cv2.imshow("Image", temp_scaled)

    elif event == cv2.EVENT_LBUTTONUP:
        # Finish drawing
        end_point = (int(x * scale_back_x), int(y * scale_back_y))
        is_drawing = False
        rectangles.append((start_point, end_point))
        cv2.rectangle(image, start_point, end_point, (0, 0, 255), 2)

        # Calculate corner points
        top_left = start_point
        bottom_right = end_point
        top_right = (end_point[0], start_point[1])
        bottom_left = (start_point[0], end_point[1])

        # Draw corner coordinates on the image
        corners = [top_left, top_right, bottom_left, bottom_right]
        for corner in corners:
            text = f"({corner[0]}, {corner[1]})"
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
            text_x = corner[0]
            text_y = corner[1] - 5 if corner[1] - 5 > 10 else corner[1] + text_size[1] + 5
            cv2.rectangle(image, (text_x, text_y - text_size[1] - 5), (text_x + text_size[0] + 5, text_y + 5), (255, 255, 255), -1)
            cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        scaled_image = cv2.resize(image, (0, 0), fx=window_scale, fy=window_scale, interpolation=cv2.INTER_LINEAR)
        cv2.imshow("Image", scaled_image)

# Display the scaled image
cv2.imshow("Image", scaled_image)
cv2.setMouseCallback("Image", draw_rectangle)

while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        # Quit the program
        break
    elif key == ord('s'):
        # Save the image with rectangles
        output_path = "output_image.jpg"
        cv2.imwrite(output_path, image)
        print(f"Image saved to {output_path}")
    elif key == ord('m'):
        # Minimize the window
        if not is_minimized:
            cv2.destroyWindow("Image")
            is_minimized = True
        else:
            cv2.imshow("Image", scaled_image)
            is_minimized = False

cv2.destroyAllWindows()


Image saved to output_image.jpg
